In [1]:
import os
import numpy as np
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns
#sns.set(style="darkgrid")
from keras.models import load_model

Using TensorFlow backend.


In [113]:
TIMESTEPS = 1000
MIN_FAULT_DURATION = 12

In [2]:
df = pd.read_excel("time_series_all.xlsx")

In [3]:
df 

,Feeder,Measurand,0,1,2,3,4,5,6,7,...,14490,14491,14492,14493,14494,14495,14496,14497,14498,14499
0,0,KV,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,B-4 S.J. ENCL.,AMPs,59.279,59.279,59.279,59.279,55.2146,55.2146,55.2146,55.2146,...,40.7447,40.7447,40.7447,50.2752,50.2752,50.2752,50.2752,62.5414,62.5414,62.5414
2,B-4 S.J. ENCL.,PF,-0.981392,-0.981392,-0.981392,-0.981392,-0.981995,-0.981995,-0.981995,-0.981995,...,-0.964221,-0.964221,-0.964221,-0.973871,-0.973871,-0.973871,-0.973871,-0.98305,-0.98305,-0.98305
3,PANKHA RD CKT-2,AMPs,84.2096,84.2096,84.2096,84.2096,71.1225,71.1225,71.1225,71.1225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PANKHA RD CKT-2,KV,68.7123,68.7123,68.7123,68.7123,68.7763,68.7763,68.7763,68.7763,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PANKHA RD CKT-2,PF,-0.993832,-0.993832,-0.993832,-0.993832,-0.994503,-0.994503,-0.994503,-0.994503,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,IGNOU S/S -3(Feeder 1),AMPs,50.6842,50.6842,50.6842,50.6842,47.0453,47.0453,47.0453,47.0453,...,33.5321,33.5321,33.5321,38.4531,38.4531,38.4531,38.4531,47.1678,47.1678,47.1678
7,IGNOU S/S -3(Feeder 1),PF,-0.996355,-0.996355,-0.996355,-0.996355,-0.996485,-0.996485,-0.996485,-0.996485,...,-0.997994,-0.997994,-0.997994,-0.998952,-0.998952,-0.998952,-0.998952,-0.998727,-0.998727,-0.998727
8,1-OKHLA (SPARE),AMPs,21.7221,21.7221,21.7221,21.7221,19.3694,19.3694,19.3694,19.3694,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1-OKHLA (SPARE),KV,66.5628,66.5628,66.5628,66.5628,66.762,66.762,66.762,66.762,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = df.dropna()

In [5]:
df = df.reset_index(drop=True)

## Resolution decreased - 1 hr

In [6]:
l = list(range(0,14000,4))
l.insert(0,'Feeder')
l.insert(1,'Measurand')

In [7]:
df = df[l]

In [8]:
df

,Feeder,Measurand,0,4,8,12,16,20,24,28,...,13960,13964,13968,13972,13976,13980,13984,13988,13992,13996
0,0,KV,0,0,0,0,0,0,0,0,...,0,0,52.9008,53.4501,53.4501,52.8092,54.2131,54.2131,52.7482,54.5793
1,B-4 S.J. ENCL.,AMPs,59.279,55.2146,53.5783,50.199,49.1715,51.5043,54.3287,53.1617,...,67.434,63.3062,57.609,54.844,52.5823,49.5275,48.9819,52.8333,56.636,57.4393
2,B-4 S.J. ENCL.,PF,-0.981392,-0.981995,-0.977454,-0.974945,-0.973284,-0.968893,-0.968026,-0.970689,...,-0.986562,-0.984885,-0.980423,-0.979735,-0.969004,-0.970104,-0.970679,-0.957499,-0.956177,-0.95474
3,IGNOU S/S -3(Feeder 1),AMPs,50.6842,47.0453,45.3992,44.8068,44.1599,42.8744,41.7501,44.4888,...,41.8592,40.0655,44.042,45.2999,38.1166,35.8424,32.9916,34.2306,34.2306,35.8432
4,IGNOU S/S -3(Feeder 1),PF,-0.996355,-0.996485,-0.99766,-0.997741,-0.99738,-0.998821,-0.998663,-0.995812,...,-0.997208,-0.995261,-0.996108,-0.995599,-0.996334,-0.996389,-0.997183,-0.994392,-0.993183,-0.993386
5,CKT 2 PAPPANKALAN,AMPs,111.635,97.6701,91.6659,88.9646,90.058,106.832,167.083,260.174,...,63.9543,73.4614,74.2103,68.9588,62.7749,59.115,54.413,54.0669,53.5102,57.0059
6,CKT 2 PAPPANKALAN,KV,70.1961,70.2288,70.6763,70.9992,71.241,70.5069,68.1071,67.2777,...,67.4672,66.9533,66.8659,67.0037,67.9309,68.2156,68.2339,68.326,68.4864,68.3384
7,CKT 2 PAPPANKALAN,PF,-0.993118,-0.990419,-0.989444,-0.987249,-0.988496,-0.992268,-0.997058,-0.994797,...,1,0.997582,0.9976,0.997402,0.997189,0.994949,0.995092,0.995932,0.996034,0.995947
8,CKT 2 MEHRAUILI I/C,AMPs,57.005,54.5396,48.8387,46.5635,45.646,45.6812,53.6347,55.7416,...,48.8769,47.1917,48.3515,54.6723,52.6404,50.4269,48.2861,51.367,51.4565,45.8522
9,CKT 2 MEHRAUILI I/C,PF,-0.988547,-0.986929,-0.989987,-0.986021,-0.984567,-0.987035,-0.972462,-0.967557,...,-0.973338,-0.977112,0.974557,-0.97611,-0.973604,-0.977668,-0.976915,-0.975031,-0.974026,-0.984179


In [10]:
df.set_index(['Feeder', 'Measurand'])

0      \
Feeder                                   Measurand                            
0                                        KV                               0   
B-4 S.J. ENCL.                           AMPs                        59.279   
                                         PF                       -0.981392   
IGNOU S/S -3(Feeder 1)                   AMPs                       50.6842   
                                         PF                       -0.996355   
CKT 2 PAPPANKALAN                        AMPs                       111.635   
                                         KV                         70.1961   
                                         PF                       -0.993118   
CKT 2 MEHRAUILI I/C                      AMPs                        57.005   
                                         PF                       -0.988547   
BHAIRON RD                               AMPs                       33.5153   
                                         PF                        0.981325   
Incomer 2 MEHRAULI CKT2                  AMPs                             0   
                                         KV                         67.1221   
                                         PF                              -1   
Incomer  OKHLA CKT 4                     AMPs                             0   
                                         KV                         34.1449   
                                         PF                              -1   
S/S-11 D-4 V.KUNJ                        AMPs                       53.0218   
                                         PF                       -0.969788   
DMRC + CARRIAPPA MARG SAINIK FORM        AMPs                       9.15866   
                                         PF                       -0.992352   
SUBHAS VIHAR-1                           AMPs                       69.4177   
                                         PF                        0.987461   
CAPBANK-4                                AMPs                             0   
                                         KV                         10.8282   
PPK SEC-10                               AMPs       External identity error   
                                         PF         External identity error   
CHWLA TRUNK                              AMPs                       53.7359   
                                         PF                        0.681104   
...                                                                     ...   
1616JANTA FLATS                          AMPs                            59   
                                         PF                               0   
T-OFF MATIYALA PANKH.                    AMPs                       23.5459   
                                         KV                         64.9818   
VENKATSWARA HOSPITAL-1                   AMPs                             0   
                                         PF                               0   
PLOT 8 DC                                AMPs                       9.39418   
                                         PF                              -1   
SEC-1 S/S-1 PUSHP VHR                    AMPs                       66.6777   
                                         PF                        0.969092   
ALI VILLAGE                              AMPs       External identity error   
                                         PF         External identity error   
J.J KHANPUR                              AMPs                       101.691   
                                         PF                       -0.960588   
LODHI ROAD I/C NO.2                      AMPs                      0.239037   
                                         KV                         33.8022   
                                         PF                              -1   
400KV MUNDKA CKT                         AMPs                       235.961   
                                         

In [188]:
feeders = []
for i in df['Feeder'].unique():
    tdf = df[df['Feeder']==i]
    measurands = [tdf[tdf['Measurand']=='AMPs'].values, tdf[tdf['Measurand']=='KV'].values, tdf[tdf['Measurand']=='PF'].values]
    if len(measurands[0])==0: continue
    feeder = []
    for m in measurands:
        if len(m) != 0:
            feeder.append(m[0][2:])
        else:
            feeder.append(-np.ones(3500))
                
    feeders.append(feeder)
feeders = np.array(feeders)

In [189]:
feeders.shape

(1392, 3, 3500)

In [190]:
def isSubArray(A, n, m, st):
    B = [0] * m
    ii = st
    j = 0
    while (ii < n and j < m): 
        if (A[ii] == B[j]): 
            ii += 1 
            j += 1 
            if (j == m): 
                return ii,True
        else: 
            ii = ii - j + 1 
            j = 0 
    return ii,False 

In [191]:
l = []
for i in range(len(feeders)):
    xx = feeders[i][0][:TIMESTEPS]
    r = isSubArray(xx.tolist(), TIMESTEPS, MIN_FAULT_DURATION,0)
    if(r[1]):
        l.append((i, r[0]))                      # i=index, r[0]='timestamp', r[1]=fault
l = pd.DataFrame(l)
l

,0,1
0,2,596
1,3,433
2,5,86
3,6,12
4,7,229
5,10,12
6,11,13
7,12,650
8,13,12
9,15,627


In [192]:
indices = list(range(len(feeders)))
for i in reversed(l[0]):
    indices.pop(i)
feeders = feeders[indices]
len(feeders)

808

# faults

In [136]:
l = []
for i in range(len(feeders)):
    xx = feeders[i][0][TIMESTEPS:]
    r = isSubArray(xx.tolist(), len(xx), MIN_FAULT_DURATION,0)
    if(r[1]):
        l.append((i, r[0]))
l = pd.DataFrame(l)

In [144]:
# l: 0=index, 1='timestamp', 2=fault
X_fault_indices = pd.DataFrame(zip(l[0], l[1], l[1].apply(
    lambda t: list(range(t-MIN_FAULT_DURATION, t+TIMESTEPS-MIN_FAULT_DURATION))))) # l[1] is already fault - TIMESTEPS
X_fault = []
for i in X_fault_indices.values:
    X_fault.append(feeders[i[0]][:,i[2]])
    
X_fault = np.array(X_fault)                 # 0 - df index for future use
#X_fault = pd.DataFrame(list(X_fault[1].values))

In [147]:
X_fault.shape #.reshape(X_fault.shape[0], X_fault.shape[1], X_fault.shape[2], 1)

(305, 3, 1000)

# no fault
selected randomly from !l

In [166]:
indices = list(range(len(feeders)))
for i in reversed(l[0]):
    indices.pop(i)
no_faults = feeders[indices]
no_faults.shape

(503, 3, 3500)

In [183]:
X_no_faults = []

for i in range(len(no_faults)):
    r = np.random.randint(feeders.shape[2] - TIMESTEPS)
    X_no_faults.append(no_faults[i][:, r:r+TIMESTEPS])
    
X_no_faults = np.array(X_no_faults)
X_no_faults.shape

(503, 3, 1000)

In [210]:
X = np.concatenate([X_no_faults, X_fault])
y = np.array([0]*len(X_no_faults) + [1]*len(X_fault))

In [211]:
X.shape, y.shape

((808, 3, 1000), (808,))

In [212]:
# Spliting into traing and testing 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [213]:
x_train.shape,x_test.shape

((646, 3, 1000), (162, 3, 1000))

In [220]:
model = Sequential()

In [221]:
model.add(LSTM(512, input_shape=(x_train.shape[1],x_train.shape[2]))) # dropout = 0.2,recurrent_dropout=0.2,

In [222]:
model.add(Dense(64, activation='relu'))

In [223]:
model.add(Dense(1, activation='sigmoid'))

In [224]:
model.compile(loss = 'binary_crossentropy' , optimizer='adam',metrics=['accuracy'])

In [225]:
history = model.fit(x_train,y_train,epochs=100,validation_split=0.2,batch_size=1000)

Train on 516 samples, validate on 130 samples
Epoch 1/100


ValueError: could not convert string to float: 'External identity error'

In [219]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('time')
plt.legend(['training', 'validation'], loc='upper right')
plt.show()


NameError: name 'history' is not defined

In [ ]:
pred = model.predict(x_train)
predict_classes = pred.argmax(axis = 1)
# Accuracy
print(sum(predict_classes==y_train)/len(y_train))

cm = confusion_matrix(y_train,predict_classes)
sns.heatmap(confusion_matrix(y_train,predict_classes),annot=True,fmt='d',cmap="Blues") 
plt.ylabel('True label')
plt.xlabel('Predicted label')

In [ ]:
pred = model.predict(x_test)
predict_classes = pred.argmax(axis = 1)
# Accuracy
print(sum(predict_classes==y_test)/len(y_test))

cm = confusion_matrix(y_test,predict_classes)
sns.heatmap(confusion_matrix(y_test,predict_classes),annot=True,fmt='d',cmap="Blues") 
plt.ylabel('True label')
plt.xlabel('Predicted label')

In [ ]:
pred